# Falta por hacer

- Imagen completa ¿?
- Transformaciones
- Imagen concatenada 

# Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm 

from radiomics import featureextractor
import SimpleITK as sitk

In [2]:
import logging

logging.getLogger('radiomics').setLevel(logging.CRITICAL) 

# Initial configuration

In [3]:
input_csv = "../../../data/data.csv"
pre_path = "../../../"

t2_features_gland_csv      = "features_t2_gland.csv"
t2_features_full_csv       = "features_t2_full.csv"
adc_features_gland_csv     = "features_adc_gland.csv"
adc_features_full_csv      = "features_adc_full.csv"
dwi_features_gland_csv     = "features_dwi_gland.csv"
dwi_features_full_csv      = "features_dwi_full.csv"
composite_features_gland_csv = "features_composite_gland.csv"
composite_features_full_csv  = "features_composite_full.csv"

extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.settings['geometryTolerance'] = 1e-4
extractor.settings['binWidth'] = 25

# Functions

In [4]:
def resample_to_reference(moving_image, reference_image, is_mask=False):
    """
    Reamuestra 'moving_image' (sitk.Image) a la geometría de 'reference_image'.
    Si 'is_mask' es True, usa interpolación 'NearestNeighbor' (para mantener etiquetas enteras).
    De lo contrario, usa interpolación 'Linear'.
    """
    resample = sitk.ResampleImageFilter()
    resample.SetReferenceImage(reference_image)
    
    if is_mask:
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    else:
        resample.SetInterpolator(sitk.sitkLinear)
    
    resampled = resample.Execute(moving_image)
    return resampled

In [5]:
def preprocess_image(image):
    """
    Aplica preprocesamiento a la imagen:
      - Corrección de ruido (denoising) usando un filtro de flujo de curvatura.
      - Corrección de no uniformidad (bias field correction) usando N4BiasFieldCorrection.
    """
    # 1. Denoising: usamos el filtro de flujo de curvatura
    curvatureFilter = sitk.CurvatureFlowImageFilter()
    curvatureFilter.SetTimeStep(0.125)
    curvatureFilter.SetNumberOfIterations(5)
    denoised = curvatureFilter.Execute(image)
    
    # 2. Corrección de no uniformidad:
    #    Se crea una máscara inicial de la imagen mediante un umbral de Otsu.
    mask = sitk.OtsuThreshold(denoised, 0, 1, 200)
    corrected = sitk.N4BiasFieldCorrection(denoised, mask)
    
    return corrected

In [6]:
def extract_radiomic_features_from_sitk(image_sitk, mask_path, patient_id, study_id, label_value, mask_type="gland"):
    """
    Extrae características radiómicas a partir de una imagen (sitk.Image) ya preprocesada.
    Permite elegir entre extraer las características sobre:
      - La glándula (usando la máscara proporcionada, mask_type="gland")
      - La imagen completa (mask_type="full")
    """
    if mask_type == "gland":
        mask_sitk = sitk.ReadImage(mask_path)
        mask_sitk = resample_to_reference(mask_sitk, image_sitk, is_mask=True)
    elif mask_type == "full":
        mask_sitk = sitk.Image(image_sitk.GetSize(), sitk.sitkUInt8)
        mask_sitk.CopyInformation(image_sitk)
        mask_sitk = sitk.Add(mask_sitk, 1)
    else:
        raise ValueError("mask_type debe ser 'gland' o 'full'")
    
    features = extractor.execute(image_sitk, mask_sitk)
    
    out_dict = {
        "patient_id": patient_id,
        "study_id": study_id,
        "label": label_value,
        "mask_type": mask_type
    }
    
    for k, v in features.items():
        if k.startswith("original_"):
            out_dict[k] = v
    
    return out_dict

In [7]:
def extract_radiomic_features(image_path, mask_path, patient_id, study_id, label_value, mask_type="gland"):
    """
    Carga la imagen desde 'image_path', le aplica preprocesamiento y extrae las características.
    """
    image_sitk = sitk.ReadImage(image_path)
    image_sitk = preprocess_image(image_sitk)
    
    return extract_radiomic_features_from_sitk(image_sitk, mask_path, patient_id, study_id, label_value, mask_type)

# "Main"

In [15]:
df = pd.read_csv(input_csv)

t2_features_gland  = []
# t2_features_full   = []
adc_features_gland = []
# adc_features_full  = []
dwi_features_gland = []
# dwi_features_full  = []
composite_features_gland = []
# composite_features_full  = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Procesando imágenes"):
    patient_id = row["patient_id"]
    study_id   = row["study_id"]
    label_val  = row["case_csPCa"]
    
    gland_mask_path = os.path.join(pre_path, row["whole_gland_path"])
    
    # ----- Procesamiento de T2 -----
    t2_img_path = os.path.join(pre_path, row["t2w_path"])
    if os.path.isfile(t2_img_path) and os.path.isfile(gland_mask_path):
        feats_t2_gland = extract_radiomic_features(
            image_path=t2_img_path,
            mask_path=gland_mask_path,
            patient_id=patient_id,
            study_id=study_id,
            label_value=label_val,
            mask_type="gland"
        )
        t2_features_gland.append(feats_t2_gland)
        
        # feats_t2_full = extract_radiomic_features(
        #     image_path=t2_img_path,
        #     mask_path=gland_mask_path,  
        #     patient_id=patient_id,
        #     study_id=study_id,
        #     label_value=label_val,
        #     mask_type="full"
        # )
        # t2_features_full.append(feats_t2_full)
    
    # ----- Procesamiento de ADC -----
    adc_img_path = os.path.join(pre_path, row["adc_path"])
    if os.path.isfile(adc_img_path) and os.path.isfile(gland_mask_path):
        feats_adc_gland = extract_radiomic_features(
            image_path=adc_img_path,
            mask_path=gland_mask_path,
            patient_id=patient_id,
            study_id=study_id,
            label_value=label_val,
            mask_type="gland"
        )
        adc_features_gland.append(feats_adc_gland)
        
        # feats_adc_full = extract_radiomic_features(
        #     image_path=adc_img_path,
        #     mask_path=gland_mask_path,
        #     patient_id=patient_id,
        #     study_id=study_id,
        #     label_value=label_val,
        #     mask_type="full"
        # )
        # adc_features_full.append(feats_adc_full)
    
    # ----- Procesamiento de DWI -----
    dwi_img_path = os.path.join(pre_path, row["hbv_path"])
    if os.path.isfile(dwi_img_path) and os.path.isfile(gland_mask_path):
        feats_dwi_gland = extract_radiomic_features(
            image_path=dwi_img_path,
            mask_path=gland_mask_path,
            patient_id=patient_id,
            study_id=study_id,
            label_value=label_val,
            mask_type="gland"
        )
        dwi_features_gland.append(feats_dwi_gland)
        
        # feats_dwi_full = extract_radiomic_features(
        #     image_path=dwi_img_path,
        #     mask_path=gland_mask_path,
        #     patient_id=patient_id,
        #     study_id=study_id,
        #     label_value=label_val,
        #     mask_type="full"
        # )
        # dwi_features_full.append(feats_dwi_full)
    
    # ----- Procesamiento de imagen compuesta -----
    # Si existen las tres modalidades, se crea una imagen compuesta promedio)
    if (os.path.isfile(t2_img_path) and os.path.isfile(adc_img_path) and os.path.isfile(dwi_img_path) 
        and os.path.isfile(gland_mask_path)):
    
        t2_image = preprocess_image(sitk.ReadImage(t2_img_path))
        adc_image = preprocess_image(sitk.ReadImage(adc_img_path))
        dwi_image = preprocess_image(sitk.ReadImage(dwi_img_path))
        
        adc_image_rs = resample_to_reference(adc_image, t2_image, is_mask=False)
        dwi_image_rs = resample_to_reference(dwi_image, t2_image, is_mask=False)
        
        t2_arr = sitk.GetArrayFromImage(t2_image).astype(np.float32)
        adc_arr = sitk.GetArrayFromImage(adc_image_rs).astype(np.float32)
        dwi_arr = sitk.GetArrayFromImage(dwi_image_rs).astype(np.float32)
        
        composite_arr = (t2_arr + adc_arr + dwi_arr) / 3.0
        composite_image = sitk.GetImageFromArray(composite_arr)
        composite_image.CopyInformation(t2_image)
        
        feats_comp_gland = extract_radiomic_features_from_sitk(
            composite_image, gland_mask_path, patient_id, study_id, label_val, mask_type="gland"
        )
        composite_features_gland.append(feats_comp_gland)
        
        # feats_comp_full = extract_radiomic_features_from_sitk(
        #     composite_image, gland_mask_path, patient_id, study_id, label_val, mask_type="full"
        # )
        # composite_features_full.append(feats_comp_full)

Procesando imágenes:   0%|▏                                                        | 4/1500 [11:42<73:00:20, 175.68s/it]


KeyboardInterrupt: 

In [30]:
t2_features_gland  = []
t2_features_full   = []
adc_features_gland = []
adc_features_full  = []
dwi_features_gland = []
dwi_features_full  = []
composite_features_gland = []
composite_features_full  = []

24

In [16]:
composite_features_gland

[{'patient_id': 10000,
  'study_id': 1000000,
  'label': 0,
  'mask_type': 'gland',
  'original_shape_Elongation': 0.8117099089996536,
  'original_shape_Flatness': 0.6388960967418924,
  'original_shape_LeastAxisLength': 29.458350250393444,
  'original_shape_MajorAxisLength': 46.1082019449155,
  'original_shape_Maximum2DDiameterColumn': array(51.75),
  'original_shape_Maximum2DDiameterRow': array(46.40253376),
  'original_shape_Maximum2DDiameterSlice': array(51.89958002),
  'original_shape_Maximum3DDiameter': array(51.89958002),
  'original_shape_MeshVolume': array(35678.30672058),
  'original_shape_MinorAxisLength': 37.42648440484501,
  'original_shape_Sphericity': array(0.75136869),
  'original_shape_SurfaceArea': array(6975.36901054),
  'original_shape_SurfaceVolumeRatio': array(0.19550729),
  'original_shape_VoxelVolume': 35708.30401048427,
  'original_firstorder_10Percentile': array(-704.39014893),
  'original_firstorder_90Percentile': array(1548.02653809),
  'original_firstorder_E